# Implementation and linear cryptanalysis of a simplified AES-like cipher 
Laboratory session 1 of *Information Security*, AY 2024/25

To Do:
- precautions for input shape or type mismatches
- proper function documentation

In [29]:
# library imports
import numpy as np
from itertools import product
from tqdm import tqdm

## Task 1
Using a programming language of your choice, implement the encryptor for a simplified
AES-like cipher.

Soooo let's do helper functions for 
- subkey generation
- substitution S
- transposition T
- linear transformation L

In [2]:
def substitution(v):
    return (2*v) % 11

def transposition(y):
    # returns copy of y with 2nd half flipped
    # assuming y has an even length
    h = len(y)//2 # halfway point
    z = np.copy(y)
    z[h:]= z[h:][::-1]
    return z

A = np.array([[2,5],[1,7]])
def linear(z):
    W = z.reshape((2,4))
    w = (A @ W) % 11
    return w.flatten()

def subkeyGen(K):
    idx = np.array([
        [0, 2, 4, 6],
        [0, 1, 2, 3],
        [0, 3, 4, 7],
        [0, 3, 5, 6],
        [0, 2, 5, 7],
        [2, 3, 4, 5]
    ])
    return K[idx]

Put everything together in an encryptor function:

In [3]:
def encryptA(u, k):
    # generate subkeys from key k
    subkeys = subkeyGen(k)
    # first subkey sum
    v = (u + np.concatenate((subkeys[0], subkeys[0]))) % 11 

    for i in range(4):
        # S: perform substitution
        y = substitution(v)
        # T: transpose
        z = transposition(y)
        # L: linear transform
        w = linear(z)
        # compute subkey sum
        v = (w + np.concatenate((subkeys[i+1], subkeys[i+1])) ) % 11

    # last iteration without linear step:
    # S: perform substitution
    y = substitution(v)
    # T: transpose
    z = transposition(y)
    # subkey sum
    v = (z + np.concatenate((subkeys[5], subkeys[5])) ) % 11
        
    return v

Test with given test message & key:

In [4]:
u = np.zeros(8)
u[0] = 1
k = u.copy()
encryptA(u,k)

array([4., 0., 0., 9., 7., 0., 0., 3.])

## Task 2
Implement the decryptor for this simplified AES-like cipher. Note that decryption is performed by the inverse blocks in reverse order. Therefore, you have to implement the inverse of each function used to encrypt the message (subkey sum, substitution, transposition and linear), taking into consideration that all the operations must be done in the field F = GF(p).

The transposition (flipping the 2nd half of the vector) is its own inverse here. For the linear transformation, 


In [5]:
def substitutionInv(v):
    return 6*v % 11

A_inv = np.array([[2, 8],[6, 10]])
def linearInv(w):
    W = w.reshape((2,4))
    z = (A_inv @ W) % 11
    return z.flatten()

def decryptA(x, k):
    # generate subkeys from key k
    subkeys = subkeyGen(k)
    
    # subkey diff
    z = (x + 11 - np.concatenate((subkeys[5], subkeys[5]))) % 11
    # T^-1: inverse transposition
    y = transposition(z)
    # S^-1: inv. substitution
    v = substitutionInv(y)

    for i in range(0, 4):
        # subkey diff
        w = (v + 11 - np.concatenate((subkeys[4-i], subkeys[4-i]))) % 11
        # L^-1: inverse linear trafo
        z = linearInv(w)
        # T^-^: inverse transposition
        y = transposition(z)
        # S^-1: inv. substitution
        v = substitutionInv(y)

    # subkey diff
    u = (v + 11 - np.concatenate((subkeys[0], subkeys[0]))) % 11
        
    return u

In [6]:
N = 100 # number of test pairs
u = np.random.randint(0, 10, size=(N, 8)) # generate random test messages
k =  np.random.randint(0, 10, size=(N, 8)) # generate random test keys
x = np.array([decryptA(encryptA(u[i], k[i]), k[i]) for i in range(N)]) # encrypt and decrypt the messages
print(np.all(u == x)) # check if all decrypted messages match the original ones

True


## Task 5
implement the encryptor for a simplified AES-like cipher with the parameters given in the
previous slides and the substitution function described by the following table


In [7]:
s_table = np.array([0, 2, 4, 8, 6, 10, 1, 3, 5, 7, 9])
s_table_inv = np.argsort(s_table)

def substitutionB(v):
    return s_table[v]

def substitutionInvB(v):
    return s_table_inv[v]
    
def encryptB(u, k):
    # generate subkeys from key k
    subkeys = subkeyGen(k)
    # first subkey sum
    v = ((u + np.concatenate((subkeys[0], subkeys[0]))) % 11).astype(int)

    for i in range(4):
        # S: substitution
        y = substitutionB(v)
        # T: transpose
        z = transposition(y)
        # L: linear transform
        w = linear(z)
        # compute subkey sum
        v = ((w + np.concatenate((subkeys[i+1], subkeys[i+1])) ) % 11).astype(int)

    # last iteration without linear step:
    # S: substitution
    y = substitutionB(v)
    # T: transpose
    z = transposition(y)
    # subkey sum
    v = ((z + np.concatenate((subkeys[5], subkeys[5])) ) % 11).astype(int)
        
    return v

def decryptB(x, k):
    # generate subkeys from key k
    subkeys = subkeyGen(k)
    
    # subkey diff
    z = ((x + 11 - np.concatenate((subkeys[5], subkeys[5]))) % 11).astype(int)
    # inv. T
    y = transposition(z)
    # inv. S
    v = substitutionInvB(y)

    for i in range(0, 4):
        # subkey diff
        w = ((v + 11 - np.concatenate((subkeys[4-i], subkeys[4-i]))) % 11).astype(int)
        # inv. L 
        z = linearInv(w)
        # inv. T
        y = transposition(z)
        # inv. S
        v = substitutionInvB(y)

    # subkey diff
    u = ((v + 11 - np.concatenate((subkeys[0], subkeys[0]))) % 11).astype(int)
        
    return u

Test encryption with given message & key:

In [8]:
u = np.zeros(8)
u[0] = 1
k = u.copy()
encryptB(u,k)

array([9, 0, 0, 0, 5, 0, 0, 6])

Test decryption:

In [9]:
N = 100 # number of test pairs
u = np.random.randint(0, 10, size=(N, 8)) # generate random test messages
k =  np.random.randint(0, 10, size=(N, 8)) # generate random test keys
x = np.array([decryptB(encryptB(u[i], k[i]), k[i]) for i in range(N)]) # encrypt and decrypt the messages
print(np.all(u == x)) # check if all decrypted messages match the original ones

True


## Task 7
implement the encryptor for a simplified AES-like cipher with the following parameters ..

In [10]:
def subkeyGenC(K):
    idx = np.array([
        [0, 1, 2, 3],
        [0, 1, 3, 2],
        [1, 2, 3, 0],
        [0, 3, 1, 2],
        [2, 3, 0, 1],
        [1, 3, 0, 2]
    ])
    return K[idx]

# modular multiplicative inverse table for GF(11)
# for v[j] = 0, we set v[j]^-1 to 0
inv_table = np.array([0, 1, 6, 4, 3, 9, 2, 8, 7, 5, 10])
def substitutionC(v):
    return 2*inv_table[v] % 11

def substitutionInvC(y):
    return inv_table[(6*y) % 11]
    
def encryptC(u, k):
    # generate subkeys from key k
    subkeys = subkeyGenC(k)
    # first subkey sum
    v = ((u + np.concatenate((subkeys[0], subkeys[0]))) % 11).astype(int)

    for i in range(4):
        # S: substitution
        y = substitutionC(v)
        # T: transpose
        z = transposition(y)
        # L: linear transform
        w = linear(z)
        # compute subkey sum
        v = ((w + np.concatenate((subkeys[i+1], subkeys[i+1])) ) % 11).astype(int)

    # last iteration without linear step:
    # S: substitution
    y = substitutionC(v)
    # T: transpose
    z = transposition(y)
    # subkey sum
    v = ((z + np.concatenate((subkeys[5], subkeys[5])) ) % 11).astype(int)
        
    return v

def decryptC(x, k):
    # generate subkeys from key k
    subkeys = subkeyGenC(k)
    
    # subkey diff
    z = ((x + 11 - np.concatenate((subkeys[5], subkeys[5]))) % 11).astype(int)
    # inv. T
    y = transposition(z)
    # inv. S
    v = substitutionInvC(y)

    for i in range(0, 4):
        # subkey diff
        w = ((v + 11 - np.concatenate((subkeys[4-i], subkeys[4-i]))) % 11).astype(int)
        # inv. L 
        z = linearInv(w)
        # inv. T
        y = transposition(z)
        # inv. S
        v = substitutionInvC(y)

    # subkey diff
    u = ((v + 11 - np.concatenate((subkeys[0], subkeys[0]))) % 11).astype(int)
        
    return u

In [11]:
u = np.zeros(8)
u[0] = 1
k = u.copy()
encryptC(u,k)

array([5, 0, 3, 2, 5, 2, 1, 1])

In [12]:
N = 100 # number of test pairs
u = np.random.randint(0, 10, size=(N, 8)) # generate random test messages
k =  np.random.randint(0, 10, size=(N, 8)) # generate random test keys
x = np.array([decryptC(encryptC(u[i], k[i]), k[i]) for i in range(N)]) # encrypt and decrypt the messages
print(np.all(u == x)) # check if all decrypted messages match the original ones

True


Task 8

In [34]:
def meet_in_the_middle_attack(encrypt_fn, decrypt_fn, P_list, C_list, keyspace_size=5):
    for pair_idx, (P, C) in enumerate(zip(P_list, C_list)):
        print(f"\nTesting Pair #{pair_idx+1}")
        P = np.array(P, dtype=int)
        C = np.array(C, dtype=int)

        forward_map = {}

        print("→ Generating forward map...")
        for k1_vals in tqdm(product(range(11), repeat=keyspace_size), total=11**keyspace_size):
            k1 = np.zeros(8, dtype=int)
            k1[:keyspace_size] = k1_vals
            mid = encrypt_fn(P, k1)
            forward_map[tuple(mid.tolist())] = k1

        print("← Searching for reverse matches...")
        for k2_vals in tqdm(product(range(11), repeat=keyspace_size), total=11**keyspace_size):
            k2 = np.zeros(8, dtype=int)
            k2[:keyspace_size] = k2_vals
            mid_candidate = decrypt_fn(C, k2)
            if tuple(mid_candidate.tolist()) in forward_map:
                k1 = forward_map[tuple(mid_candidate.tolist())]
                print(f"\nMatch Found!")
                print(f"Plaintext: {P}")
                print(f"Ciphertext: {C}")
                print(f"k1: {k1}")
                print(f"k2: {k2}")

                # Verify correctness
                test = encrypt_fn(encrypt_fn(P, k1), k2)
                print("Double encryption result:", test)
                print("Valid:", np.all(test == C))
                break
        else:
            print("No match found for this pair.")

In [35]:
plaintexts = [
    [4, 1, 6, 10, 2, 3, 5, 10],
    [10, 5, 4, 4, 7, 3, 2, 0],
    [2, 6, 8, 0, 6, 8, 10, 9],
    [3, 7, 2, 10, 1, 6, 9, 0],
    [5, 1, 6, 3, 10, 8, 8, 10],
]

ciphertexts = [
    [2, 1, 4, 0, 6, 7, 5, 5],
    [2, 8, 6, 2, 6, 2, 10, 0],
    [5, 5, 1, 4, 10, 2, 9, 2],
    [3, 8, 10, 10, 10, 9, 7, 8],
    [10, 0, 8, 10, 2, 10, 2, 2],
]

# Now run it
meet_in_the_middle_attack(encryptB, decryptB, plaintexts, ciphertexts, keyspace_size=5)


Testing Pair #1
→ Generating forward map...


100%|██████████| 161051/161051 [00:26<00:00, 6087.85it/s]


← Searching for reverse matches...


  1%|          | 1708/161051 [00:00<00:31, 5053.39it/s]



Match Found!
Plaintext: [ 4  1  6 10  2  3  5 10]
Ciphertext: [2 1 4 0 6 7 5 5]
k1: [1 4 1 9 9 0 0 0]
k2: [0 1 3 1 3 0 0 0]
Double encryption result: [2 1 4 0 6 7 5 5]
Valid: True

Testing Pair #2
→ Generating forward map...


...


Match Found!
Plaintext: [ 3  7  2 10  1  6  9  0]
Ciphertext: [ 3  8 10 10 10  9  7  8]
k1: [1 2 5 9 7 0 0 0]
k2: [ 0  0 10  3  6  0  0  0]
Double encryption result: [ 3  8 10 10 10  9  7  8]
Valid: True

Testing Pair #5
→ Generating forward map...


100%|██████████| 161051/161051 [00:22<00:00, 7144.81it/s]


← Searching for reverse matches...


  0%|          | 654/161051 [00:00<00:27, 5928.69it/s]


Match Found!
Plaintext: [ 5  1  6  3 10  8  8 10]
Ciphertext: [10  0  8 10  2 10  2  2]
k1: [10  3  6  4  8  0  0  0]
k2: [0 0 5 4 5 0 0 0]
Double encryption result: [10  0  8 10  2 10  2  2]
Valid: True
